Module 13:
Eliyahou Amsellem
Brian Livian
Mazal-Tov

# Introduction

The goal of this assignment is to migrate the tables from the Chinoock PostgreSQL database to a new MongoDB. Ultimately Python will be used to extract the data from MongoDB but only to be used in the format of a pandas dataframe that resembles the original tables from the Postgres database. 

Steps to Completion:



1.   Create AWS Server and Database (issues with Postgres previsouly)
2.   Open load the Chinook Database
3.   Connect Python Notebook to database
4.   Query the tables and save the results
5.   Create a function that turns query results into dataframe
6.   Change the format of the dataframe to bson
7. Create MongoDB account, cluster, and database
8.   Load the dataframes into the MongoDB
9.   Extract data from MongoDB and create dataframes similar to the tables Invoice and Customer

# PART 1

To extract data from pgAdmin and upload it to Mongo DB we had to take a few different steps. We first had to establish a connection with pgAdmin to access the chinook data. Then we used queries to extract only the Invoice and Customer tables' data. With that data in the form of a pandas dataframe we had to write a method that transformed our data into a BSON format to work with Mongo DB. Once properly formatted we made a connection with Mongo DB, created a database to upload our data, and uploaded the data into collections corresponding to the chinook tables'.

In [5]:
import pandas as pd
import psycopg2
import sqlalchemy
import matplotlib.pyplot as plt
#establish connection to pgAdmin so we can access chinook data
conn = psycopg2.connect(
    host="localhost",
    database="chinook",
    user="postgres",
    password="1015")

In [6]:
# Create a cursor object
cur = conn.cursor()

In [97]:
#start the connection and retreive the Inovice and Customer tables
#save the tables' data into pandas dataframe
curs = conn.cursor()
curs.execute("ROLLBACK")
conn.commit()
customerdf = pd.read_sql_query("""SELECT * FROM "Customer" """, conn)

cur.execute("""SELECT * FROM "Invoice" """)
invoicedf = pd.read_sql_query("""SELECT * FROM "Invoice" """, conn)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [60]:
# convert dataframe to dict
customerdict = customerdf.to_dict()
invoicedict = invoicedf.to_dict()

In [61]:
# input columns of dataframes into lists
customercolumns= []
invoicecolumns = []
for column in customerdict:
    customercolumns.append(column)
for column in invoicedict:
    invoicecolumns.append(column)

In [62]:
# function to convert dictionaries into bson format for mongodb
# bson will be a list of dictionaries, each dictionary representing a row in the df
# function takes inputs of count, columnlist (customer/invoice), number of rows (to know when to stop the function), and dictionary (customer/invoice)
count = 0
bson = []
def function(count, columnlist, numofrows, dictionary):
    x = []
    # for each customer's value in key:value pair (count), append list x 
    # create a dictionary by zipping 'columnlist' key with 'x' value
    for i in columnlist:
        x.append(dictionary[i][count])
    bson.append(dict(zip(columnlist, x)))
    count = count + 1
    # continue algorithm for all rows in respective df
    if count < numofrows:
        function(count, columnlist, numofrows, dictionary)
    return bson


In [63]:
# run function with inputs for customer table
# reset 'bson' below, otherwise the list can be appended indefinitely
bson = []
customerbson = function(0, customercolumns, 59, customerdict)

# demonstrate function was successful by displaying length of list = number of rows in df
len(customerbson)

59

In [85]:
# same process for invoice
bson = []
invoicebson = function(0, invoicecolumns, 412, invoicedict)

# demonstrate function was successful by displaying length of list = number of rows in df
len(invoicebson)

412

In [74]:
import pymongo
from pymongo import MongoClient
#establish connection with mMongo DB
client = MongoClient()

In [75]:
#create AIM5001 database
db = client['AIM5001']
db = client.AIM5001
#Include Customer and Invoice documents
Customer = db["Customer"]
Invoice = db["Invoice"]

In [76]:
#insert data from BSON format of Customer table into mongo created document

for i in range(len(customerbson)):
    Customer.insert(customerbson[i])

<ipython-input-76-0ec29c4da0a7>:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  Customer.insert(customerbson[i])


In [77]:
#insert data from BSON format of Invoice table into mongo created documents

for i in range(len(invoicebson)):
    Invoice.insert(invoicebson[i])

<ipython-input-77-c76595034291>:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  Invoice.insert(invoicebson[i])


In [96]:
# get a list of databases on your local server
client.list_database_names()

['AIM5001', 'admin', 'config', 'local']

In [86]:
# retrieve + display all documents in a collection
#Put this here to verify the information was accurate and properly alligned
for document in Customer.find():
print (document)

{'_id': ObjectId('6085e3777db5a57f290c3cef'), 'CustomerId': 1, 'FirstName': 'Luís', 'LastName': 'Gonçalves', 'Company': 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Address': 'Av. Brigadeiro Faria Lima, 2170', 'City': 'São José dos Campos', 'State': 'SP', 'Country': 'Brazil', 'PostalCode': '12227-000', 'Phone': '+55 (12) 3923-5555', 'Fax': '+55 (12) 3923-5566', 'Email': 'luisg@embraer.com.br', 'SupportRepId': 3}
{'_id': ObjectId('6085e3777db5a57f290c3cf0'), 'CustomerId': 2, 'FirstName': 'Leonie', 'LastName': 'Köhler', 'Company': None, 'Address': 'Theodor-Heuss-Straße 34', 'City': 'Stuttgart', 'State': None, 'Country': 'Germany', 'PostalCode': '70174', 'Phone': '+49 0711 2842222', 'Fax': None, 'Email': 'leonekohler@surfeu.de', 'SupportRepId': 5}
{'_id': ObjectId('6085e3777db5a57f290c3cf1'), 'CustomerId': 3, 'FirstName': 'François', 'LastName': 'Tremblay', 'Company': None, 'Address': '1498 rue Bélanger', 'City': 'Montréal', 'State': 'QC', 'Country': 'Canada', 'PostalCode': 'H2G 1

# PART 2

This process was fairly simple and convenient. From within the Jupyter environment we establish a connection with Mongo DB. Once connected we describe which collection we would like to retrieve data from (AIM5001). Once we know what data we would like to extract we can upload it into a dataframe autonomously with pandas the dataframe method.

In [93]:
client = MongoClient()
#select database
newdb = client['AIM5001']
#select the collection within the database
test = newdb.Invoice
#convert entire collection to Pandas dataframe
Invoicedf = pd.DataFrame(list(test.find()))
Invoicedf

,_id,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,6085e3837db5a57f290c3d2a,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6085e3837db5a57f290c3d2b,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,6085e3837db5a57f290c3d2c,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,6085e3837db5a57f290c3d2d,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,6085e3837db5a57f290c3d2e,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...,...
407,6085e3837db5a57f290c3ec1,408,25,2013-12-05,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,6085e3837db5a57f290c3ec2,409,29,2013-12-06,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,6085e3837db5a57f290c3ec3,410,35,2013-12-09,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,6085e3837db5a57f290c3ec4,411,44,2013-12-14,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [95]:
#select the collection within the database
test2 = newdb.Customer
#convert entire collection to Pandas dataframe
Customerdf = pd.DataFrame(list(test.find()))
Customerdf

,_id,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,6085e3837db5a57f290c3d2a,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,6085e3837db5a57f290c3d2b,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,6085e3837db5a57f290c3d2c,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,6085e3837db5a57f290c3d2d,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,6085e3837db5a57f290c3d2e,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...,...
407,6085e3837db5a57f290c3ec1,408,25,2013-12-05,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,6085e3837db5a57f290c3ec2,409,29,2013-12-06,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,6085e3837db5a57f290c3ec3,410,35,2013-12-09,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,6085e3837db5a57f290c3ec4,411,44,2013-12-14,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86
